## Convective/Stratiform identification for 3D Reflectivity from derived dBZ of WRF Simulations. 

**Output the Convective/Stratiform (C/S) Mask information to CONUS dBZ data.**

**For [High Resolution WRF Simulations of the Current and Future Climate of North America](https://rda.ucar.edu/datasets/ds612.0/).**

**Hungjui Yu 20211022**

In [176]:
import sys
# from shutil import copyfile
import time
import datetime as dt
import pytz
from netCDF4 import (Dataset, MFDataset)
import numpy as np
# from scipy.interpolate import interpn
import xarray as xr
import pandas as pd
import wrf
# from wrf import (getvar, vinterp, interplevel, extract_times, destagger, to_np, latlon_coords, get_cartopy,
#                  cartopy_xlim, cartopy_ylim)

# import cartopy.crs as ccrs
# from cartopy.feature import NaturalEarthFeature
# import matplotlib as mpl
# import matplotlib.pyplot as plt

import conv_stra_sep as cs_sep

**Set input files paths and names:**

In [177]:
def set_input_names(file_date):

    file_path_1_conus = '/gpfs/fs1/collections/rda/data/ds612.0'
    file_path_1_dbz = '/glade/scratch/hungjui/DATA_WRF_CONUS_1_dBZ_v1.0'
    file_path_2 = '/' + wrf_sim_type # '/CTRL3D'
    file_path_3 = '/{}'.format(file_date.strftime('%Y'))

    file_names = dict( dbz = file_path_1_dbz
                           + file_path_2 
                           + '/20130913'# file_path_3 
                           + '/wrf3d_d01_' + wrf_sim_type[0:-2] + '_dbz_{}.nc'.format(file_date.strftime('%Y%m%d'))
                       # , Z = file_path_1 
                       #     + file_path_2 
                       #     + file_path_3 
                       #     + '/wrf3d_d01_' + wrf_sim_type[0:-2] + '_Z_{}.nc'.format(file_date.strftime('%Y%m%d'))
                     )
    
    return file_names

**Function: Convective/Stratiform separation:**

In [178]:
def CS_separation(refl, lat, lon):

    ## Make sure the array for the masking is in numpy array format (speed issue):
    # refl = dbz_sigmalev.data
    # lat = np.array(dbz_sigmalev.XLAT)
    # lon = np.array(dbz_sigmalev.XLONG)

    # start = time.time()

    ## Convective/Stratiform separation:
    cs, cc, bkgnd = cs_sep.conv_stra_sep(refl, lat, lon)

    # end = time.time()
    # print("RUNTIME：%f SEC" % (end - start))
    # print("RUNTIME：%f MIN" % ((end - start)/60))
    # print("RUNTIME：%f HOUR" % ((end - start)/3600))
    
    cs[np.where(refl <= 0.0)] = np.nan
    cs[np.where(np.isnan(refl))] = -1
    
    return cs

### Main Function:

In [179]:
def main_function(file_date_time):
    
    ## Set file datetime:
    # file_date_time = dt.datetime(2013, 9, 13, 0, 0, 0, tzinfo=pytz.utc)
    print('Processing: {}'.format(file_date_time.strftime('%Y%m%d')), end=': ')
    
    ## Set input files paths and names:
    file_name_dict = set_input_names(file_date_time)

    ## Get the 3-hourly time list:
    nc_wrf_dbz = Dataset(file_name_dict['dbz'], mode='r')
    wrf_3hour_list = wrf.extract_times(nc_wrf_dbz, timeidx=wrf.ALL_TIMES, meta=False, do_xtime=False)
    nc_wrf_dbz.close()
    
    ## Open dBZ data array and append calculated data:
    ds_wrf_dbz = xr.open_dataset(file_name_dict['dbz'])
    
    ## Set sigma level index:
    interp_vertical_lev = 12 # number in levels.
    
    for hi in range(len(wrf_3hour_list)):
        
        print(str(hi) + ' | ', end=' ')

        ## Get dBZ data:
        da_wrf_dbz = ds_wrf_dbz['dBZ'].isel(Time=hi)
        
        ## Get dBZ data at specified sigma level:
        dbz_sigmalev = da_wrf_dbz[interp_vertical_lev,:,:]
        
        ## Convective/Stratiform Separation:
        ## !!! Make sure the array for the masking is in numpy array format (speed issue) !!!
        CS_mask_single = CS_separation( dbz_sigmalev.data
                                      , np.array(dbz_sigmalev.XLAT)
                                      , np.array(dbz_sigmalev.XLONG)
                                      )
        
        ## Stack the CS mask according to hours:
        if ( hi == 0 ):
            # CS_mask = CS_mask_single
            CS_mask = np.expand_dims(CS_mask_single, axis=0)
        else:
            # CS_mask = xr.concat([CS_mask, CS_mask_single], dim='TimeDim')
            CS_mask = np.append(CS_mask, np.expand_dims(CS_mask_single, axis=0), axis=0)
            
        # print(type(CS_mask))
        # print(CS_mask.shape)

        
    ## Add CS mask to dBZ dataset:
    ds_wrf_dbz['CS_mask'] = (['Time', 'south_north', 'west_east'], CS_mask)
    
    ds_wrf_dbz.close()
    
    return ds_wrf_dbz
    
    ## Add CS mask to dBZ netcdf file directly:
    # nc_wrf_dbz = Dataset(file_name_dict['dbz'], mode='a')



### Main Program:

In [180]:
start = time.time()


## WRF Model Simulation Category:
wrf_sim_type = 'CTRL3D'
# wrf_sim_type = 'PGW3D'

## Loop through a period:
target_date_range = pd.date_range(start='2013-9-13', end='2013-9-13', tz=pytz.utc)

for dayi in target_date_range:
    
    ds_wrf_dbz = main_function(dayi)
    
    ## Set output file path and name:
    output_path_1 = '/glade/u/home/hungjui/2scratch/DATA_WRF_CONUS_1_dBZ_v1.0/' + wrf_sim_type
    output_path_2 = '/20130913'
    # output_path_2 = '/{}'.format(file_date_time.strftime('%Y'))
    # output_file_name = '/wrf3d_d01_CTRL_dbz_CSmask_20130913.nc'
    output_file_name = '/wrf3d_d01_CTRL_dbz_20130913.nc'
    # output_file_name = set_output_name(file_date_time)
    ds_wrf_dbz.to_netcdf(output_path_1 + output_path_2 + output_file_name, 'a')

    
end = time.time()

print("RUNTIME：%f SEC" % (end - start))
print("RUNTIME：%f MIN" % ((end - start)/60))
print("RUNTIME：%f HOUR" % ((end - start)/3600))
    

Processing: 20130913: 0 |  1 |  2 |  3 |  4 |  5 |  6 |  7 |  

In [175]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


<font color='teal'>**Supplement Codes:**</font>

In [115]:
# from netCDF4 import (Dataset, MFDataset)
# nc_wrf_dbz = Dataset( '/glade/scratch/hungjui/DATA_WRF_CONUS_1_dBZ_v1.0/CTRL3D/20130913/wrf3d_d01_CTRL_dbz_20130913.nc'
#                     , mode='a')
# # nc_wrf_dbz.createDimension('test_dim', 1)
# print(nc_wrf_dbz)
# nc_wrf_dbz.close()